# Numpy Broadcasting -  advanced manipulation of arrays

In [ ]:
%matplotlib inline
from pylab import *
from numpy import *

## Advanced manipulation of shapes

From the manual: 

The term broadcasting describes how numpy treats arrays with different shapes during arithmetic operations. Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they have compatible shapes. Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python. It does this without making needless copies of data and usually leads to efficient algorithm implementations. There are, however, cases where broadcasting is a bad idea because it leads to inefficient use of memory that slows computation.

In [ ]:
x = arange(15)
print(x)
print(x*2)
print(x+2)

The operations `x*2` or `x+2` are the simplest broadcasting operation you can think of. The number `2` is extended to an array of the same dimension of `x` and then multiplied or summed **element wise** (all operations, in numpy, are element wise by default).

We can think of the scalar `2` being stretched during the arithmetic operation into an array with the same shape as `x`. The new elements are simply copies of the original scalar. The stretching analogy is only conceptual. NumPy is smart enough to use the original scalar value without actually making copies, so that **broadcasting operations are as memory and computationally efficient as possible**.


## General Broadcasting Rules

When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing dimensions, and works its way forward. Two dimensions are compatible when

- they are equal, or
- one of them is 1

![Broadcast](http://www.astroml.org/_images/fig_broadcast_visual_1.png)

If these conditions are not met, a `ValueError: frames are not aligned` exception is thrown, indicating that the arrays have incompatible shapes. The size of the resulting array is the maximum size along each dimension of the input arrays.

Arrays do not need to have the same number of dimensions. For example, if you have a 256x256x3 array of RGB values, and you want to scale each color in the image by a different value, you can multiply the image by a one-dimensional array with 3 values. Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:

    Image  (3d array): 256 x 256 x 3
    Scale  (1d array):             3
    Result (3d array): 256 x 256 x 3
    
Here are some more examples:

    A      (2d array):  5 x 4
    B      (1d array):      1
    Result (2d array):  5 x 4

    A      (2d array):  5 x 4
    B      (1d array):      4
    Result (2d array):  5 x 4

    A      (3d array):  15 x 3 x 5
    B      (3d array):  15 x 1 x 5
    Result (3d array):  15 x 3 x 5

    A      (3d array):  15 x 3 x 5
    B      (2d array):       3 x 5
    Result (3d array):  15 x 3 x 5

    A      (3d array):  15 x 3 x 5
    B      (2d array):       3 x 1
    Result (3d array):  15 x 3 x 5
    

## Fast two dimensional plotting, and grid creation

One of the most powerful way to exploit this, is when constructing tensor product spaces, or writing and plotting functions of two variables:

In [ ]:
x = linspace(0,1,111)
y = linspace(10,11,77)

What if we wanted sin(2*pi*x)*sin(2*pi*y) for all combinations of x and y?

First method:

In [ ]:
# This creates two equal matrices, containing the x and y values repeated along rows and columns
X,Y = meshgrid(x,y)

Z = sin(2*pi*X)*sin(4*pi*Y)

fig, ax = plt.subplots()
cnt = ax.contour(X,Y,Z)

If we know that we are going to exploit the tensor product structure, i.e., we can write `Z = f(x)*g(y)`, it is more convenient (and faster from the numerical point of view) to use broadcasting:

In [ ]:
# Evaluate cos and sin in the x and y directions
sinx = sin(2*pi*x)
siny = sin(4*pi*y)

# Then broadcast sinx and siny
Z = sinx*siny[:,newaxis]

fig, ax = plt.subplots()
cnt = ax.contour(x,y,Z) # Notice small x and y

What just happened?

In [ ]:
print(siny.shape)
print(siny[:,newaxis].shape)

We added a new shape axis to the `siny` array. This allows us to broadcast it with the `sinx` array, to obtain  a `77x111` matrix, representing the values of our function at the `x` (111 points) and `y` (77 points) positions.

The difficult thing to get right at first is the order of the indices. Plots in matplotlib are obtained by running faster with x, then with y, and so on. 

This is called **lexycographical** ordering. If we look at the matrix entries `Z[i,j]`, the index `j` runs over `x`, and the index `i` runs over `y`. Another equivalent way to obtain the picture above is to use `einsum`:

In [ ]:
Z = einsum('i,j', siny, sinx) # sum over repeated indices: no repetitions = broadcasting

fig, ax = plt.subplots()
cnt = ax.imshow(Z) # Notice small x and y